# RAG system about the magical worlds 'Harry Potter' and 'Middle Earth'

## Importing libraries

In [1]:
import os
from decouple import config
from dotenv import load_dotenv

from langchain import PromptTemplate
from langchain_community.document_loaders import (
    PyPDFLoader,
    DirectoryLoader,
    TextLoader,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain_qdrant import QdrantVectorStore
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
from langchain_ollama import OllamaLLM, ChatOllama

In [2]:
load_dotenv()

True

## Preparing a vector database, in this project I use Qdrant

In [3]:
# from langchain.vectorstores import FAISS

In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

In [5]:
QDRANT_URL = config("QDRANT_URL")
QDRANT_API_KEY = config("QDRANT_API_KEY")

In [6]:
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY, prefer_grpc=False)

In [7]:
collection_name = "literature_collection"

### Creating a new collection in vectorstore if we don't have any

In [8]:
# client.create_collection(
#     collection_name="literature_collection",
#     vectors_config=(VectorParams(size=768, distance=Distance.COSINE))
# )

In [ ]:
# scroll_result, _ = client.scroll(
#     collection_name="literature_collection",
#     limit=100_000,
#     with_payload=True
# )

# sources = set()
# for point in scroll_result:
#     source = point.payload.get("metadata").get('source')
#     if source:
#         sources.add(source)

# for s in sorted(sources):
#     print(s)

In [ ]:
# scroll_result[0].payload.get("metadata").get('source')

### PDF directory loader

In [ ]:
# loader = DirectoryLoader('data/', glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader)

### TXT directory loader

In [ ]:
# loader = DirectoryLoader('data/txt_data/', glob="**/*.txt", show_progress=True, loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})

In [ ]:
# documents = loader.load()

### Splitter for text

In [ ]:
# splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
# split_documents = splitter.split_documents(documents)

In [ ]:
# for i, doc in enumerate(split_documents[0:0]):  # fiest 5 as an example
#     print(f"\n Document {i + 1}:")
#     print(doc.page_content)
#     print(doc.metadata)

## Importing the embeddings model from HuggingFace. I'm using sentence-transformers/all-mpnet-base-v2

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

## Initializing Qdrant vector store

In [10]:
vectorstore = QdrantVectorStore(
    embedding=embeddings, client=client, collection_name=collection_name
)

In [ ]:
# vectorstore = FAISS.from_documents(split_documents, embeddings)

### Adding splitted documents to vectorstore

In [ ]:
# vectorstore.add_documents(split_documents)

## Creating a retriever

In [35]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

## Importing Mistral-7B-instruct model from Ollama

In [36]:
llm_ollama = OllamaLLM(
    model="mistral:instruct", 
    temperature=0.2, 
    top_k=50, 
    top_p=1, 
    verbose=True,
)

In [37]:
chat_llm_ollama = ChatOllama(
        model="mistral:instruct", 
        temperature=0.2,
        verbose=True, 
        top_k=50,
        top_p=1)

## Importing local model Mistral-7B-instruct from gguf file

In [48]:
local_llm_name = "mistral-7b-instruct-v0.2.Q5_K_S.gguf"

In [49]:
config = {
    "max_new_tokens": 2048,
    "context_length": 2048,
    "repetition_penalty": 1.1,
    "temperature": 0.2,
    "top_k": 50,
    "top_p": 1,
    "stream": True,
    "threads": int(os.cpu_count() / 2),
}

In [50]:
local_llm = CTransformers(model=local_llm, model_type="mistral", lib="avx2", **config)

## Creating a prompt template with my context and question

In [14]:
prompt_template = """
You're a literature expert on the magic world of 'Harry Potter' and 'Middle Earth'.
Your task is to answer the user's question using the information from provided context, book quotes.

Context:{context}
Question:{question}

Please be accurate and do not make up facts. If you don't know the answer or it is unclear, explain what is known and what is unclear.
"""
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [15]:
chain_type_kwargs = {"prompt": prompt}

## Using Ollama Mistral-instruct model

### Creating a chain to generate the answer

In [38]:
qa = RetrievalQA.from_chain_type(
    llm=llm_ollama,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True,
    verbose=True,
)

#### 1 Question

In [39]:
query="Tell me about the text on the One Ring."


In [40]:
result = qa.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [41]:
print("\nAnswer:")
print(result["result"])

print("\nSources:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n Source {i + 1}:")
    print(doc.page_content[:500])
    print("Source:", doc.metadata.get("source", "unknown"))


Answer:
 The text inscribed on the One Ring is a verse from Elven-lore, specifically from J.R.R. Tolkien's 'The Lord of the Rings'. The full verse reads: "One Ring to rule them all, One Ring to find them, One Ring to bring them all and in the Darkness bind them. In the Land of Mordor where the Shadows lie."

This verse is a part of a longer poem that describes the history and power of the Rings of Power. The One Ring was said to be the first of the Seven Rings forged by Sauron, though it was given to Durin III, King of Khazad-dûm, by Elven-smiths themselves.

The text on the ring is written in a flowing script made up of lines of fire that appear bright and remote, as if from a great depth. The exact meaning of the symbols "#n_88" within the verse is not explicitly explained in the provided context, but it seems to be part of the Elven-lore associated with the ring.

It's important to note that this information comes from 'The Hobbit', and while Tolkien expanded on the lore of Middle 

#### 2 Question

In [42]:
query = "Tell about the story of Deathly Hallows"

In [43]:
result = qa.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [44]:
print("\nAnswer:")
print(result["result"])

print("\nSources:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n Source {i + 1}:")
    print(doc.page_content[:500])
    print("Source:", doc.metadata.get("source", "unknown"))


Answer:
 The Deathly Hallows refer to three magical artifacts mentioned in the tale "The Tale of the Three Brothers." These objects are said to give their possessor mastery over death. The story is not explicitly detailed within the Harry Potter series, but it is referenced multiple times.

1. The Invisibility Cloak: This cloak was owned by the first brother and allows its wearer to become invisible. It plays a significant role in the series, as Harry inherits this cloak from his father.

2. The Resurrection Stone: This stone is said to bring back the dead for one minute. It is one of the Hallows that Voldemort seeks, and Dumbledore believes it could potentially bring back Harry's parents. However, its exact role in the story remains unclear.

3. The Elder Wand (also known as the Deathstick or the wand of Destiny): This wand is said to be the most powerful wand ever made and chooses its master. It is another object that Voldemort seeks, and it plays a crucial role in the series' event

#### 3 Question

In [45]:
query = "What animal was Harry's patronus?"

In [46]:
result = qa.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [47]:
print("\nAnswer:")
print(result["result"])

print("\nSources:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n Source {i + 1}:")
    print(doc.page_content[:500])
    print("Source:", doc.metadata.get("source", "unknown"))


Answer:
 The animal that was Harry Potter's Patronus was a stag. This is confirmed in the quote provided where Harry whispers "Prongs," which is a nickname for his friend Sirius Black, who transforms into a large black dog (an Animagus) and whose Patronus is also a stag. The stag is a symbol of purity and courage in the 'Harry Potter' series, often associated with Dumbledore and Sirius Black.

Sources:

 Source 1:
The Patronus turned. It was cantering back toward Harry across the still surface of the water. It wasn't a horse. It wasn't a unicorn, either. It was a stag. It was shining brightly as the moon above ... it was coming back to him....
　　It stopped on the bank. Its hooves made no mark on the soft ground as it stared at Harry with its large, silver eyes. Slowly, it bowed its antlered head. And Harry realized... "Prongs, "he whispered.
Source: data\txt_data\harry_potter\Harry Potter and the Prisoner of Azkaban.txt

 Source 2:
'Harry, I think I'm doing it!' yelled Seamus, who had

#### 4 question

In [48]:
query = "Describe Harry and Ginny's relationships throught books"

In [49]:
result = qa.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [50]:
print("\nAnswer:")
print(result["result"])

print("\nSources:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n Source {i + 1}:")
    print(doc.page_content[:500])
    print("Source:", doc.metadata.get("source", "unknown"))


Answer:
 In the Harry Potter series by J.K. Rowling, the relationship between Harry Potter and Ginny Weasley evolves over time, starting as a close friendship and eventually developing into romantic feelings. However, their relationship faces numerous challenges due to external factors, particularly Harry's responsibilities as the Chosen One in his fight against Voldemort.

Throughout the books, it is clear that Ginny has had feelings for Harry since she was a young girl. This becomes more apparent during their shared summer at the Burrow in Harry Potter and the Chamber of Secrets, where they bond over Quidditch, teasing Ron, and laughing about Bill and his adventures with Phlegm. However, Harry views Ginny as more of a younger sister due to their close relationship.

As they grow older, their feelings for each other become more complex. In Harry Potter and the Order of the Phoenix, Harry begins to realize that he has deeper feelings for Ginny, but he is hesitant to act on them due to

#### 5 Question

In [51]:
query = "Describe the appearence of Harry Potter and Ginny Weasley."

In [52]:
result = qa.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [53]:
print("\nAnswer:")
print(result["result"])

print("\nSources:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n Source {i + 1}:")
    print(doc.page_content[:500])
    print("Source:", doc.metadata.get("source", "unknown"))


Answer:
 In the provided context, neither Harry Potter nor Ginny Weasley's appearance is explicitly described. However, we can infer some details based on their association with other characters.

Harry Potter is not described in this specific context, but we know from previous books that he has black hair and glasses. He is also often noted to be quite small for his age.

Ginny Weasley, Harry's cousin, is part of the Weasley family, which includes six brothers (Fred, George, Ron, Percy, Bill, and Charlie) and one sister (Ginny). In this context, Ginny is described as being in a picture with her brother Ron. The picture does not show their hair color, but we know from other books that the Weasley children have red hair. So, it can be inferred that Ginny also has red hair like her siblings.

The description of Ginny's appearance beyond this is not provided in the given context.

Sources:

 Source 1:
haired; Mr. Weasley, kind-faced, balding, his spectacles a little awry; Mad-Eye, battle

## Using ChatOllama

In [54]:
qa_chat = RetrievalQA.from_chain_type(
    llm=chat_llm_ollama,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True,
    verbose=True,
)

In [55]:
query="Tell me about the text on the One Ring."

In [56]:
result = qa_chat.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [57]:
print("\nAnswer:")
print(result["result"])

print("\nSources:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n Source {i + 1}:")
    print(doc.page_content[:500])
    print("Source:", doc.metadata.get("source", "unknown"))


Answer:
 The text on the One Ring is a verse from Elven-lore, specifically from J.R.R. Tolkien's 'The Lord of the Rings'. It reads: "One Ring to rule them all, One Ring to find them, One Ring to bring them all and in the Darkness bind them."

This verse is actually part of a longer poem that describes the history and power of the Rings of Power. The One Ring was said to be the first of the Seven Rings forged by Sauron, and it was given to Durin III, King of Khazad-dûm (Khazadum is Dwarvish for 'Durin's Hall'), by Elven-smiths. However, it is unclear whether Sauron had a direct hand in its creation or not.

The text on the ring is written in a flowing script of fine lines that shone piercingly bright, as if from a great depth. The letters are #n_88, but their meaning or significance within the verse is not explicitly stated in the provided context. It's also important to note that the full poem describes three other rings: Three for the Elven-kings, Seven for the Dwarf-lords, and Nine 